# Mission Parameters

In [1]:
import geemap
import os
import ee
import numpy as np
import matplotlib.pyplot as plt
import csv
import datetime
import pickle 
params = pickle.load(open( "params.p", "rb" ))
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
print(color.BOLD + 'Define Radial Zone and Time Period to Collect Fire Data' + color.END)
longitude = input(color.UNDERLINE + 'General Longitude of Interest (float):' + color.END)
try:
    val = float(longitude)
except ValueError:
    print("That's not a number! Rerun the cell and enter a GPS longitude coordinate (float)")   
latitude = input(color.UNDERLINE + 'General Latitude of Interest (float):' + color.END)
try:
    val = float(latitude)
except ValueError:
    print("That's not a number! Rerun the cell and enter a GPS latitude coordinate (float)")
radius_of_interest_meters = input(color.UNDERLINE + 'Radius of Interest (int, meters):' + color.END)
try:
    val = int(radius_of_interest_meters)
except ValueError:
    print("That's not a number! Rerun the cell and enter a flight altitude value (int, meters)")
start = input(color.UNDERLINE + 'Start date for fire data collection (yyyy-mm-dd):' + color.END)
try:
    datetime.datetime.strptime(start, "%Y-%m-%d").date()
except ValueError:
    print("That's not a date! Rerun the cell and enter a start date using the yyyy-mm-dd format")
end = input(color.UNDERLINE + 'End date for fire data collection (yyyy-mm-dd):' + color.END)
try:
    datetime.datetime.strptime(end, "%Y-%m-%d").date()
except ValueError:
    print("That's not a date! Rerun the cell and enter a start date using the yyyy-mm-dd format")
print('')
print('')
print(color.BOLD + 'Define Flight Altitude (x meter max for Unreal Simulation, x meter max for Tello Drone)' + color.END)
altitude = input(color.UNDERLINE + 'Flight Altitude (int):' + color.END)
try:
    val = int(altitude)
except ValueError:
    print("That's not a number! Rerun the cell and enter a flight altitude (int)") 
altitude = int(altitude)
if altitude >= params['max_alt']: 
    print("That altitude is too high! Rerun the cell and enter a flight altitude within the given range (int)") 

Define Radial Zone and Time Period to Collect Fire Data
General Longitude of Interest (float):-121.8811
General Latitude of Interest (float):37.3352
Radius of Interest (int, meters):40000
Start date for fire data collection (yyyy-mm-dd):2021-04-25
End date for fire data collection (yyyy-mm-dd):2021-04-26


Define Flight Altitude (x meter max for Unreal Simulation, x meter max for Tello Drone)
Flight Altitude (int):29


In [2]:
#requirements
import geemap
import os
import ee
import numpy as np
import matplotlib.pyplot as plt
import csv

longitude = float(longitude)
latitude = float(latitude)
radius_of_interest_meters = int(radius_of_interest_meters)
start = str(start).strip()
end = str(end).strip()



#create basemap for SJ
Map = geemap.Map(center=[latitude, longitude], zoom=12)
Map.add_basemap('HYBRID')
area_of_interest = ee.Geometry.Point([longitude, latitude]).buffer(radius_of_interest_meters)
goes_16_data = ee.ImageCollection('NOAA/GOES/16/FDCF').filterDate(start, end).filterBounds(area_of_interest)
goes_17_data = ee.ImageCollection('NOAA/GOES/17/FDCF').filterDate(start, end).filterBounds(area_of_interest)
fire_mask_codes = [10, 30, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35]
confidence_values = [1.0, 1.0, 0.9, 0.9, 0.8, 0.8, 0.5, 0.5, 0.3, 0.3, 0.1, 0.1]
default_confidence_value = 0
def map_from_mask_codes_to_confidence_values(image):   
    return image.clip(area_of_interest).remap(fire_mask_codes, confidence_values, default_confidence_value)
goes_16_confidence = goes_16_data.select(['Mask']).map(map_from_mask_codes_to_confidence_values)
goes_17_confidence = goes_17_data.select(['Mask']).map(map_from_mask_codes_to_confidence_values)
goes_16_max_confidence = goes_16_confidence.reduce(ee.Reducer.max())
goes_17_max_confidence = goes_17_confidence.reduce(ee.Reducer.max())
Map.centerObject(area_of_interest, 9)
affected_area_palette = ['white', 'yellow', 'orange', 'red', 'purple'];
combined_confidence = ee.ImageCollection([goes_16_max_confidence,
goes_17_max_confidence]).reduce(ee.Reducer.min())
kernell = ee.Kernel.square(2000, 'meters', True)
smoothed_confidence = combined_confidence.reduceNeighborhood(
    reducer=ee.Reducer.mean(),
    kernel= kernell,
    optimization= 'boxcar')
high_confidence = smoothed_confidence.gt(0.6)
affected_areas = high_confidence.reduceToVectors(
    scale= 200, 
    maxPixels= 1e10,
    geometry= area_of_interest).filter(ee.Filter.eq('label', 1));
affected_areas_outline = ee.Image().byte().paint(featureCollection= affected_areas,
width= 2)
def smooth(feature):
    max_error_meters = 500
    return ee.Feature(feature).simplify(max_error_meters)
affected_areas_smoothed = ee.FeatureCollection(affected_areas).map(smooth)
affected_areas_smoothed_outline = ee.Image().byte().paint(featureCollection= affected_areas_smoothed,
width= 2)
Map.addLayer(area_of_interest, {'opacity':0.2}, "Area of Interest")
Map.addLayer(smoothed_confidence,
{'opacity': 0.3, 'min': 0, 'max': 1, 'palette': affected_area_palette},
'Smoothed confidence')
Map.addLayer(high_confidence,
{'opacity': 0.3, 'min': 0, 'max': 1, 'palette': affected_area_palette},
'High confidence')
Map.addLayer(affected_areas_smoothed_outline,
{'palette': 'purple'},
'Smoothed affected areas', True, 0.3)
print('Wait for the fire detection data to load . . .')
print(color.BOLD + 'Step 1: Use the Draw a Marker tool to define flight destination (fire area, etc.)' + color.END)
print(color.BOLD + 'Step 2: Use the Draw a Rectangle tool to define general flight area' + color.END)
Map

Wait for the fire detection data to load . . .
Step 1: Use the Draw a Marker tool to define flight destination (fire area, etc.)
Step 2: Use the Draw a Rectangle tool to define general flight area


Map(center=[37.33524103448547, -121.88109856880833], controls=(WidgetControl(options=['position', 'transparent…

#### Output Info & Leave Notebook 

In [3]:
#draw region on map 
feature = Map.draw_features
roi = feature[1].geometry()

collection = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2020-01-01', '2021-01-01').filterBounds(roi).sort('CLOUD_COVER').first()
img = collection.select(['B4', 'B3', 'B2'])
#img = trueColor.reduce(ee.Reducer.mean())

Map.centerObject(roi, 13)
Map.addLayer(collection,{'min':0, 'max':3000, 'bands': ['B4', 'B3', 'B2'], 'opacity':0.5}, 'Sentinel')
#~250k max pixels (wxh)
#shape is height, width
rgb_img = geemap.ee_to_numpy(img, region=roi)
from PIL import Image
pixels = np.asarray(rgb_img)
rgb_img_test = (255*(pixels - pixels.min())/(pixels.max() - pixels.min())).astype('uint8')
#plt.imshow(rgb_img_test)
#plt.show()
#array to image and save locally (need tif for prediction script)
from PIL import Image
image = Image.fromarray(rgb_img_test)
Image.fromarray(rgb_img_test).save('satellite.tif')

params['sat_img'] = rgb_img_test
params['altitude'] = altitude
pickle.dump(params, open( "params.p", "wb" ))

props = roi.getInfo()
coords = props['coordinates']
flat_list = [item for sublist in coords for item in sublist]
coords = flat_list[:4]
coords = str(coords).replace('[',"").replace("]","").split(', ')
fire = feature[0].geometry()
fireinfo = fire.getInfo()
coords_fire = fireinfo['coordinates']
coords_fire = [str(i) for i in coords_fire]
with open('coordinates.csv', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(coords)
    write.writerow(coords_fire)